In [1]:
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
import os
import csv
# import pyarrow

# import duckdb

# conn = duckdb.connect('mydb.db') # create an in-memory database

In [12]:
# from Bio import SeqIO

In [2]:
conn.sql("SHOW ALL TABLES")

NameError: name 'conn' is not defined

In [3]:
pwd

'/global/scratch/users/empchase/CiberVI_SpikeIn/wustl_outputs/read1'

In [4]:
path = '/global/scratch/users/empchase/CiberVI_SpikeIn/wustl_outputs/read1'

directory = os.fsencode(path)
AD1files = []
RPTR1files = []

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith('.fastq'):
#         print (type(filename))
#         read1files.append(filename)
#         print(type(filename))
        x = filename.split('_')
#         print(x)
        if 'AD' in x:
            AD1files.append(filename)
        elif 'RPTR' in x:
            RPTR1files.append(filename)
            
print(AD1files)
print(RPTR1files)

['Staller_AD_4_30_1_MVS_0001_I1_AACGCAATTC_CGTATCCGTA_S323_R1_001.fastq', 'Staller_AD_4_30_0_MVS_0003_I1_TGGACTCGAA_TTCCGTATTG_S325_R1_001.fastq']
['Staller_RPTR_4_30_1_MVS_0002_I1_CGTCGCTAAG_AATACCAGGA_S324_R1_001.fastq', 'Staller_RPTR_4_30_0_MVS_0004_I1_TTAGGTTGTC_TCTTCAACCG_S326_R1_001.fastq', 'shortStaller_RPTR_4_30_0_MVS_0004_I1_TTAGGTTGTC_TCTTCAACCG_S326_R1_001.fastq']


In [5]:
def complement(seq):
    complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A', 'N':'N', 'X':'X'} 
    bases = list(seq) 
    bases = [complement[base] for base in bases] 
    return ''.join(bases)
def reverse_complement(s):
        return complement(s[::-1])

In [10]:
def getmid(seq, pre, post, bclen):
    # seq = the sequence to parse
    # pre = substring that precedes piece of interest
    # post = substring that follows piece of interest
    # returns piece of interest

    re_key = pre + "(.*)"+ post 
    poi_search = re.search(re_key, seq)
#     print(poi_search)


    if poi_search is None:
        #the barcode will be called X
        poi = "X"
        
        #then we search for which restriction site had the error
        #regex for the bc we want to ignore
        w = ".{"+str(bclen)+"}" 
#         print(w)
        pre_re = pre + w + "(.{7})"
        pre_search = re.search(pre_re, seq)
#         print(pre_search)
        post_re = "(\w{7})" + w + post
        post_search = re.search(post_re, seq)
#         print(post_search)
        
        if pre_search is None and post_search is None:
            a = 'X'
            z = 'X'
        elif pre_search is None:
            a = post_search.group(1)
            z = post
        elif post_search is None:
            z = pre_search.group(1)
            a = pre
            
    else:
        poi = poi_search.group(1)
        a = pre
        z = post
    
    return poi, a, z


#putative consensus sequences ***reverse complement of snapgene***
adp = 'CGGGCCC'#7 bp ; beforeAD barcode in read1
adf = 'GGCGCGC' #7bp ; after AD barcode in read1

rpp = 'AGCGGCC' #7bp ; before rptr barcode in read1
rpf = 'CTCGAGT' #7 bp ; after rptr barcode in read1


# function that just looks for bc
def bc_finder(readfile,bc_pre, bc_post, bc_len):

    # make lists of reads
    seqlist = []
    
    with open(readfile, 'r') as fin:
        for line in fin:
            if line.startswith('@'):
                #look at next line to get read sequence, add to list
                seq = next(fin)
                seq = seq.strip()
                seqlist.append(seq)

    
    #make lists of BCs from list of reads
    bc_list = []
    bc_lens = []
    q_list = []
    
    prex_list = []
    prex_len = []
    postx_list = []
    postx_len = []
       
    for read in seqlist:
   
        bc, prex, postx = getmid(read, bc_pre, bc_post, bc_len)
        bc = reverse_complement(bc) #return reverse complement
        bc_list.append(bc)
        bcl = len(bc)
        bc_lens.append(bcl)
        
        prex_list.append(prex)
        prex_len.append(len(prex))
        postx_list.append(postx)
        postx_len.append(len(postx))
        if bcl == bc_len:
            q_list.append(1)
        else:
            q_list.append(0)
    #         print(bc)

               
    # make the df
    
#     print(len(bc_list))
#     print(len(bc_lens))
#     print(len(q_list))
#     print(len(prex_list))
#     print(len(prex_len))
#     print(len(postx_list))
#     print(len(postx_len))
    
    BC_dict = {"BCs":bc_list, "Length":bc_lens,"Qual":q_list, 
               "prex":prex_list, 'prexl':prex_len,
               'postx':postx_list, 'postxl':postx_len} #not lengths but matches to rpp and rpf
    BC_df = pd.DataFrame.from_dict(BC_dict)
    
    return BC_df

In [12]:
t1 = bc_finder(RPTR1files[2], rpp, rpf, 14)
t1

9
9
9
9
9
9
9


,BCs,Length,Qual,prex,prexl,postx,postxl
0,TGTGGGCATGTTAG,14,1,AGCGGCC,7,CTCGAGT,7
1,TGGAGAAAGAAAAT,14,1,AGCGGCC,7,CTCGAGT,7
2,X,1,0,NGCGGCC,7,CTCGAGT,7
3,X,1,0,NGCGGCC,7,CTCGAGT,7
4,TGTGGGCATGTTAG,14,1,AGCGGCC,7,CTCGAGT,7
5,X,1,0,X,1,X,1
6,X,1,0,X,1,X,1
7,X,1,0,X,1,X,1
8,CTCAATCGACAATT,14,1,AGCGGCC,7,CTCGAGT,7


In [18]:
# options = [1]
t2 = t1[(t1['prexl']!=1) & (t1['Length']==1)]
t2

,BCs,Length,Qual,prex,prexl,postx,postxl
2,X,1,0,NGCGGCC,7,CTCGAGT,7
3,X,1,0,NGCGGCC,7,CTCGAGT,7


In [7]:
# conn.sql("""CREATE TABLE TBB_UNION AS SELECT * FROM read_parquet('A10TBB_FILUNION.parquet')""")

In [7]:
#read in as df and create dictionary
tbbdf = conn.sql("""SELECT * FROM TBB_UNION""").df()
tbbdf.head()

,Tile,ADbc,RPTRbc,TBB
0,GATTTGCAAGGTAAATTCTTGGCTGCTCCATTGGAAGAGAATCCAA...,TTTATCCTCGG,AAAAAATTACTTGG,GATTTGCAAGGTAAATTCTTGGCTGCTCCATTGGAAGAGAATCCAA...
1,GACCCATATATGTCTGCTCCAAATTCTACTGCATTTACTGCTTTGA...,TGAAATGGTAA,CAAAAAGCTATGGT,GACCCATATATGTCTGCTCCAAATTCTACTGCATTTACTGCTTTGA...
2,GCTTTGGCTGGTGATCCATGGTATCCATTGTTTCCACAAGATGATC...,TGGTAATGAGA,TTTAAATGGCTAAT,GCTTTGGCTGGTGATCCATGGTATCCATTGTTTCCACAAGATGATC...
3,GATTATCATTCTCAAGCTACTGCTGACTTTGTCTTGTATCCACAAG...,AAGGTCCGAGG,AACAAATACAGAGT,GATTATCATTCTCAAGCTACTGCTGACTTTGTCTTGTATCCACAAG...
4,GATATTGAAGCTAGATCATTCTATGTCTTTGGTTCTCCAGTTTCTC...,CCCCGCCGTCG,TCTAGGTTTCACCC,GATATTGAAGCTAGATCATTCTATGTCTTTGGTTCTCCAGTTTCTC...


In [8]:
#dictionary ~ {RPTRbc: TBB}
RPTRdict = dict(zip(tbbdf.RPTRbc, tbbdf.TBB))

In [9]:
len(RPTRdict) #12 million RPTR BCs

12089998

In [10]:
def analyze_bcs (df, bc_len, bcdict):
    # df = barcode containing df, parsed from fastq
    # bc_len = int, expected barcode length
    # tbb_dictkey = str, either 'ADbc' or 'RPTRbc'
    
    print('total reads')
    print(df.shape[0]) #how many total reads
    
    cls = df[df['Length']== bc_len] #cl = correct length
    print('BCs of correct length')
    print(cls.shape[0])
    
    print('% BCs of correct length')
    print (cls.shape[0]/df.shape[0])
    
    
    cl_covdf = cls['BCs'].value_counts().to_frame().reset_index() #df of BC coverage
    print('Unique BCs')
    print(cl_covdf.shape[0])
    
    bcs = cl_covdf['index'].tolist()
#     print(bcs[:5])
    
    
    print('# BC matches to A10 deep seq map')

    matchlist = [] #list of TBBs that match the BCs
    matches = 0
    for x in bcs:
        try:
            tbb = bcdict[x]
        except KeyError:
            matchlist.append(0)
        else:
            matchlist.append(tbb)
            matches += 1
            

            
    cl_covdf['TBB'] = matchlist
    
    print(matches)
    
    print('% unique BCs matched')
    print(matches/cl_covdf.shape[0])
    
    
    print()
    
    return cl_covdf
    

In [26]:
# make df out of each fastq file
Alist = []
Rlist = []

# for file in AD1files:
#     x = bc_finder(file, adp, adf, 11)
#     Alist.append(x)


for file in RPTR1files:
    x = bc_finder(file, rpp, rpf, 14)
    Rlist.append(x)

In [47]:
RPTR_4_30_0 = analyze_bcs (Rlist[1], 14, RPTRdict)
RPTR_4_30_0.head()

total reads
690741
BCs of correct length
223325
% BCs of correct length
0.3233122110892505
Unique BCs
678
# BC matches to A10 deep seq map
51
% unique BCs matched
0.0752212389380531



,index,BCs,TBB
0,TGGAGAAAGAAAAT,84837,0
1,TGTGGGCATGTTAG,73485,GGTAATTATTTGTTCTCTGCTGCTGCTCATGTTCATAGATCTTCTA...
2,GAAACACTAAATTC,22077,0
3,CTCAATCGACAATT,16978,TTTCCAACTGCTAATATTCCATCTGAGTTCTTTAATTCATTTACTA...
4,CCTGACTGATAGGG,13128,0


In [51]:
rptrmatch0 = RPTR_4_30_0[RPTR_4_30_0['TBB']!=0]
rptrmatch0

,index,BCs,TBB
1,TGTGGGCATGTTAG,73485,GGTAATTATTTGTTCTCTGCTGCTGCTCATGTTCATAGATCTTCTA...
3,CTCAATCGACAATT,16978,TTTCCAACTGCTAATATTCCATCTGAGTTCTTTAATTCATTTACTA...
5,TTTAACGCATGAGT,8661,CCATCTCAATTGCAGAATACTACTCCAAATTCTCCACAACCAACTA...
12,TGGTGAAAGAAAAT,103,GGTGATCCATGGTATCCATTGTTTCCACCAGAAGATCAATTTGAAG...
25,TGTGGGCTTGTTAG,41,TTTCAAGGTGAACCATTGATGAATAATTCTAATGATTGGTTCTCTT...
48,GAAACACTAAATTA,17,AAGATGGTCTTGGAAGATTTGCAAGGTTTGTCTGACTTTGCTGCTG...
51,ATTAACGCATGAGT,16,GGTGATTTGTCTACTTTGTCATTTGACTTTGATATGTCTGCTAATG...
61,TGTGGGCATGTTTG,14,TCTTTGCAGAATCAAAGAGTTGCTAATATTATTCAAGCTACTGGTC...
64,GAAACACTTAATTC,13,AGACCATGTTTGAGATGTCCATTGTCTTTGCCAGGTAAGAGAGCTA...
74,TGGAGGAAGAAAAT,12,AGAGCTTCTAGAGATTTGGGTCCAATTAAATATGATCCAACTAATC...


In [48]:
RPTR_4_30_1 = analyze_bcs (Rlist[0], 14, RPTRdict)
RPTR_4_30_1.head()

total reads
688729
BCs of correct length
401302
% BCs of correct length
0.5826703972099331
Unique BCs
1124
# BC matches to A10 deep seq map
75
% unique BCs matched
0.06672597864768683



,index,BCs,TBB
0,TGGAGAAAGAAAAT,171407,0
1,TGTGGGCATGTTAG,138423,GGTAATTATTTGTTCTCTGCTGCTGCTCATGTTCATAGATCTTCTA...
2,CTCAATCGACAATT,28896,TTTCCAACTGCTAATATTCCATCTGAGTTCTTTAATTCATTTACTA...
3,CCTGACTGATAGGG,24265,0
4,TTTAACGCATGAGT,15664,CCATCTCAATTGCAGAATACTACTCCAAATTCTCCACAACCAACTA...


In [52]:
rptrmatch1 = RPTR_4_30_1[RPTR_4_30_1['TBB']!=0]
rptrmatch1

,index,BCs,TBB
1,TGTGGGCATGTTAG,138423,GGTAATTATTTGTTCTCTGCTGCTGCTCATGTTCATAGATCTTCTA...
2,CTCAATCGACAATT,28896,TTTCCAACTGCTAATATTCCATCTGAGTTCTTTAATTCATTTACTA...
4,TTTAACGCATGAGT,15664,CCATCTCAATTGCAGAATACTACTCCAAATTCTCCACAACCAACTA...
10,TGGTGAAAGAAAAT,227,GGTGATCCATGGTATCCATTGTTTCCACCAGAAGATCAATTTGAAG...
22,TGTGGGCTTGTTAG,106,TTTCAAGGTGAACCATTGATGAATAATTCTAATGATTGGTTCTCTT...
...,...,...,...
993,TGGATAAATATAAT,1,GCTGATAGAATTTCTGAAAATTCTAAGATGATTAAGACTGAAATTT...
1013,TGTTGACATTTTAT,1,TTGTCTGCTTCTACTTCTTCTACTGCTTTGCAAAGAAGAGGTCATC...
1024,TTTAACGCATGTGG,1,CCAGAACAAGAAACTAAGATTCCAGATGCTGTTTATGCTCCAGCTA...
1041,TGTGTGCAAGTTAG,1,GGTAATTATAATCAACACTTTGGTGCTGCTGGTATTATTTCTTCTC...


In [53]:
rptrdata_merge = rptrmatch1.merge(rptrmatch0, how='outer', on='index', suffixes=('_1', '_0'))

rptrdata_merge

,index,BCs_1,TBB_1,BCs_0,TBB_0
0,TGTGGGCATGTTAG,138423.0,GGTAATTATTTGTTCTCTGCTGCTGCTCATGTTCATAGATCTTCTA...,73485.0,GGTAATTATTTGTTCTCTGCTGCTGCTCATGTTCATAGATCTTCTA...
1,CTCAATCGACAATT,28896.0,TTTCCAACTGCTAATATTCCATCTGAGTTCTTTAATTCATTTACTA...,16978.0,TTTCCAACTGCTAATATTCCATCTGAGTTCTTTAATTCATTTACTA...
2,TTTAACGCATGAGT,15664.0,CCATCTCAATTGCAGAATACTACTCCAAATTCTCCACAACCAACTA...,8661.0,CCATCTCAATTGCAGAATACTACTCCAAATTCTCCACAACCAACTA...
3,TGGTGAAAGAAAAT,227.0,GGTGATCCATGGTATCCATTGTTTCCACCAGAAGATCAATTTGAAG...,103.0,GGTGATCCATGGTATCCATTGTTTCCACCAGAAGATCAATTTGAAG...
4,TGTGGGCTTGTTAG,106.0,TTTCAAGGTGAACCATTGATGAATAATTCTAATGATTGGTTCTCTT...,41.0,TTTCAAGGTGAACCATTGATGAATAATTCTAATGATTGGTTCTCTT...
...,...,...,...,...,...
90,TGTCGAAAGATTAG,NaN,NaN,1.0,TTCTCTGGTAACTTCTCTGCTGCTGGTGAATCTTTGTTTGATTTGC...
91,TGGTGTCATGTTAT,NaN,NaN,1.0,GGTCATTCTATTAATACTTCTGCTTTGACTAATAGATTTACTCCAC...
92,AGTAGAAAGAAACA,NaN,NaN,1.0,TCTGATGATACTATTTCTGCTGCTACTACTTCTGCTTCTTCTCCAG...
93,TGGAGTAAGAAGAT,NaN,NaN,1.0,TCTCCATCTATTTATAATGAATCTCCAGACTTTACTGATACATTTG...


In [55]:
rptrdata_merge[rptrdata_merge.isnull()any(axis=1)]

SyntaxError: invalid syntax. Perhaps you forgot a comma? (3967041304.py, line 1)

In [56]:
# rptrdata_merge.to_csv('cibervi_rptrcounts.csv')

In [57]:
pwd

'/global/scratch/users/empchase/CiberVI_SpikeIn/wustl_outputs/read1'

Testing 9/11

In [5]:
testdict = {'A':'apple', 'B':'banana', 'c':['cat', 'kitten'], 'd':'dog'}
testdf = pd.DataFrame.from_dict(testdict)
testdf

,A,B,c,d
0,apple,banana,cat,dog
1,apple,banana,kitten,dog


In [6]:
dict(zip(testdf.A, testdf.c))

{'apple': 'kitten'}